In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
raw_df = pd.read_csv('/content/drive/MyDrive/MSBD/Foundation of Data Analytics/Individual project/train.csv')
raw_df

,id,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
0,0,3556.0,2489.0,265.19,77.53,176.55,0.00,4.20,307.91,52,0,7515.0,1
1,1,1906.0,134.0,1442.61,551.90,876.07,112.10,168.15,1735.48,20,1,1756.0,0
2,2,1586.0,71.0,1332.74,684.20,655.26,244.95,216.52,1820.04,28,1,1311.0,0
3,3,683.0,94.0,419.23,255.80,162.17,72.05,44.68,538.22,55,1,1443.0,0
4,4,1032.0,71.0,1102.72,480.27,625.30,188.78,130.77,1427.97,28,1,1542.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,82,626.0,68.0,1771.57,666.99,1117.48,360.21,118.84,2306.82,42,1,1521.0,0
83,83,1237.0,71.0,1348.53,428.09,924.69,120.02,48.67,1524.78,56,0,1345.0,0
84,84,634.0,1002.0,1300.00,558.00,724.00,67.00,105.00,1484.26,34,0,2926.0,1
85,85,112.0,884.0,942.83,378.49,567.06,116.77,31.81,1104.59,33,1,2352.0,1


In [3]:
col_drop = ['Age', 'Sex 0M1F', 'id']
raw_df.drop(col_drop, axis=1, inplace=True)
raw_df

,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Mono CD64+MFI (cells/ul),label
0,3556.0,2489.0,265.19,77.53,176.55,0.00,4.20,307.91,7515.0,1
1,1906.0,134.0,1442.61,551.90,876.07,112.10,168.15,1735.48,1756.0,0
2,1586.0,71.0,1332.74,684.20,655.26,244.95,216.52,1820.04,1311.0,0
3,683.0,94.0,419.23,255.80,162.17,72.05,44.68,538.22,1443.0,0
4,1032.0,71.0,1102.72,480.27,625.30,188.78,130.77,1427.97,1542.0,0
...,...,...,...,...,...,...,...,...,...,...
82,626.0,68.0,1771.57,666.99,1117.48,360.21,118.84,2306.82,1521.0,0
83,1237.0,71.0,1348.53,428.09,924.69,120.02,48.67,1524.78,1345.0,0
84,634.0,1002.0,1300.00,558.00,724.00,67.00,105.00,1484.26,2926.0,1
85,112.0,884.0,942.83,378.49,567.06,116.77,31.81,1104.59,2352.0,1


Fill Null using interpolation.

In [4]:
neg_raw_df = raw_df[raw_df['label']==0]
neg_raw_df = neg_raw_df.interpolate('spline', order=4)
df = pd.concat([raw_df[raw_df['label']==1], neg_raw_df])

Select non-overlapping set of feature.

In [6]:
X = df.drop(['label'], axis=1)
cols = ['MO HLADR+ MFI (cells/ul)', 'Neu CD64+MFI (cells/ul)', 'CD8+T (cells/ul)', 
        'NK (cells/ul)', 'CD19+ (cells/ul)', 'CD45+ (cells/ul)']
X = X[cols]
y = df['label']

Choose the best model (and hyperparameters).  
SVM and LR are selected by score.

In [7]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut, KFold, GridSearchCV

clf = LogisticRegression(solver='saga', max_iter=2000)
# clf = make_pipeline(StandardScaler(), SVC())
# clf = RandomForestClassifier()
# clf = make_pipeline(StandardScaler(), KNeighborsClassifier())



C_range = np.logspace(-10, 10, 21)
gamma_range = np.logspace(-9, -5, 5)

regularizer = ['l1', 'l2','elasticnet', 'none']
l1_ratio_range = np.linspace(0.0, 1.0, 11)

max_depth_range = list(np.linspace(2, 5, 4)) + [None]
min_samples_split_range = np.linspace(2, 5, 4, dtype=np.int32)
max_features = ['sqrt', 'log2']
ccp_alpha_range = np.linspace(0.00, 0.02, 5)

n_neighbors_range = np.linspace(2, 5, 4, dtype=np.int32)
weights = ['uniform', 'distance'] 


# param_grid = dict(svc__gamma=gamma_range, svc__C=C_range)
param_grid = dict(C = C_range, penalty = regularizer, l1_ratio = l1_ratio_range)
# param_grid = dict(max_depth = max_depth_range,
#                   min_samples_split = min_samples_split_range,
#                   max_features = max_features,
#                   ccp_alpha = ccp_alpha_range)
# param_grid = dict(kneighborsclassifier__n_neighbors=n_neighbors_range, kneighborsclassifier__weights=weights)

cv=KFold(5)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv, n_jobs=-1)
grid.fit(X, y)

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

The best parameters are {'C': 0.01, 'l1_ratio': 0.1, 'penalty': 'elasticnet'} with a score of 0.91


Check the accuracy of SVM and logistic regression on the entire training set. LR picked to overfit less.

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(C=1e8, gamma=1e-6))
clf.fit(X, y)
clf.score(X, y)

0.9310344827586207

In [9]:
clf = LogisticRegression(solver='saga', max_iter=2000, C=0.01, penalty='elasticnet', l1_ratio=0.1)
clf.fit(X, y)
clf.score(X, y)

0.9195402298850575

Generate prediction of test set.

In [10]:
raw_df_test = pd.read_csv('/content/drive/MyDrive/MSBD/Foundation of Data Analytics/Individual project/test.csv')
df_test = raw_df_test.drop(col_drop, axis=1)

# X_test = df_test
X_test = df_test[cols]

y_test = clf.predict(X_test)
for l in y_test:
    print(l)

0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
1
1
0
0
1
0
0
1
0
1
1
0
1
0
0
0
1
1
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
1
